googleドライブ

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

huggingface

In [ ]:
!pip install huggingface_hub

from google.colab import userdata
from huggingface_hub import login

# Secretsからトークンを取得
HF_TOKEN = userdata.get('HF_TOKEN')
login(HF_TOKEN)

ライブラリ

In [ ]:
%%capture
!pip install unsloth
!pip uninstall unsloth -y && pip install --upgrade --no-cache-dir "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
!pip install -U torch
!pip install -U peft

パラメータ

In [ ]:
from unsloth import FastLanguageModel
from peft import PeftModel
import torch
import json
from tqdm import tqdm
import re

import json
import torch
from tqdm import tqdm
from huggingface_hub import login
from unsloth import FastLanguageModel
from peft import PeftModel
import gc


#model_id = "llm-jp/llm-jp-3-13b"
model_id = "/content/drive/MyDrive/program/code_translation/base_model/llm-jp-3-1.8b"
model_adapter = "test_2"
#adapter_id = "puwaer/llm-jp-3-13b-it_lora"
adapter_id = f"/content/drive/MyDrive/program/code_translation/model_adapter/{model_adapter}"

#benchmark_data = "./elyza-tasks-100-TV_test.jsonl"
benchmark_data = "/content/drive/MyDrive/program/code_translation/data/elyza-tasks-100-TV_test.jsonl"


dtype = None                                                # Noneにしておけば自動で設定
load_in_4bit = True                                         # 今回は13Bモデルを扱うためTrue

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name=model_id,
    dtype=dtype,
    load_in_4bit=load_in_4bit,
    trust_remote_code=True,
)

#model = PeftModel.from_pretrained(model, adapter_id, token = HF_TOKEN)
model = PeftModel.from_pretrained(model, adapter_id, token = HF_TOKEN)


datasets = []
with open(benchmark_data, "r") as f:
    item = ""
    for line in f:
      line = line.strip()
      item += line
      if item.endswith("}"):
        datasets.append(json.loads(item))
        item = ""

推論

In [ ]:
# 推論するためにモデルのモードを変更
FastLanguageModel.for_inference(model)

results = []
for dt in tqdm(datasets):
  input = dt["input"]

  prompt = f"""### 指示\n{input}\n### 回答\n"""

  inputs = tokenizer([prompt], return_tensors = "pt").to(model.device)

  outputs = model.generate(**inputs, max_new_tokens = 512, use_cache = True, do_sample=False, repetition_penalty=1.2)
  prediction = tokenizer.decode(outputs[0], skip_special_tokens=True).split('\n### 回答')[-1]

  results.append({"task_id": dt["task_id"], "input": input, "output": prediction})

保存

In [ ]:
# 結果をjsonlで保存。

# ここではadapter_idを元にファイル名を決定しているが、ファイル名は任意で問題なし。
json_file_id = re.sub(".*/", "", adapter_id)
with open(f"/content/{json_file_id}_output.jsonl", 'w', encoding='utf-8') as f:
    for result in results:
        json.dump(result, f, ensure_ascii=False)
        f.write('\n')